<a href="https://colab.research.google.com/github/MadmanMarble/MadmanMarble/blob/main/dataprep_imagedem_stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
install.packages(c('raster','sf','fasterize'))
install.packages("googledrive")
install.packages("dplyr")
install.packages("devtools")

devtools:::install_github("gearslaboratory/gdalUtils")

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘proxy’, ‘e1071’, ‘sp’, ‘Rcpp’, ‘terra’, ‘classInt’, ‘s2’, ‘units’, ‘wk’, ‘RcppArmadillo’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘httpuv’, ‘xtable’, ‘sourcetools’, ‘later’, ‘promises’, ‘shiny’, ‘downlit’, ‘htmlwidgets’, ‘miniUI’, ‘pkgdown’, ‘profvis’, ‘urlchecker’





R.methodsS3 (NA -> 1.8.2 ) [CRAN]
R.oo        (NA -> 1.26.0) [CRAN]
iterators   (NA -> 1.0.14) [CRAN]
R.utils     (NA -> 2.12.3) [CRAN]
foreach     (NA -> 1.5.2 ) [CRAN]


Skipping 1 packages not available: rgdal

Installing 6 packages: R.methodsS3, R.oo, iterators, rgdal, R.utils, foreach

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘rgdal’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”


── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpuGY4YP/remotes3e235de68d1/gearslaboratory-gdalUtils-8872aaf/DESCRIPTION’ ... OK
* preparing ‘gdalUtils’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘gdalUtils_2.0.3.2.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in i.p(...):
“installation of package ‘/tmp/RtmpuGY4YP/file3e27d2fd5a9/gdalUtils_2.0.3.2.tar.gz’ had non-zero exit status”


In [2]:
library(raster)
library(sf)
library(fasterize)
library(magrittr)
library(googledrive)
library(parallel)
library(data.table)
library(stringr)
library(dplyr)
# library(gdalUtils)

drive_auth(use_oob = TRUE, cache = FALSE)

Loading required package: sp

Linking to GEOS 3.11.1, GDAL 3.6.4, PROJ 9.1.1; sf_use_s2() is TRUE


Attaching package: ‘fasterize’


The following object is masked from ‘package:graphics’:

    plot


The following object is masked from ‘package:base’:

    plot



Attaching package: ‘magrittr’


The following object is masked from ‘package:raster’:

    extract



Attaching package: ‘data.table’


The following object is masked from ‘package:raster’:

    shift



Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:raster’:

    intersect, select, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/v2/auth?client_id=603366585132-frjlouoa3s2ono25d2l9ukvhlsrlnr7k.app

Enter authorization code: eyJjb2RlIjoiNC8wQWVhWVNIQU5DeHpUSi1XcWFZXzZObUJFZ2ZvakptdDBpbU9KamEzS2U4MjlZczMzOFZOcUg1a1hRTUpYVlh0eDc5UFE1USIsInN0YXRlIjoiYzMyMzFkOGM5OTIwMmRlYmZkYmI0Y2MyMDg3ZDhmZDUifQ==


In [3]:
dir.create("/content/folder")
dir.create("/content/folder/planet")
dir.create("/content/folder/planet/utm4")
dir.create("/content/folder/planet/utm4/2018")
dir.create("/content/folder/planet/utm4/2019")
dir.create("/content/folder/planet/utm4/2020")
dir.create("/content/folder/planet/utm4/dem")

# dir.create("/content/folder/planet/utm5")
# dir.create("/content/folder/planet/utm5/2018")
# dir.create("/content/folder/planet/utm5/2019")
# dir.create("/content/folder/planet/utm5/2020")
# dir.create("/content/folder/planet/utm5/dem")

In [4]:
cl <- makeCluster(8)

# planet utm4 2018
# https://drive.google.com/drive/folders/1PndP3cbvQNabol-1-mbRgnsUE-jp8FhD?usp=drive_link
jp_folder = "1PndP3cbvQNabol-1-mbRgnsUE-jp8FhD"
folder_id = drive_get(as_id(jp_folder)) # get folder ID
print(folder_id) #print folder ID

# Find files in the folder
files = drive_ls(folder_id)

# Loop through directories and download files inside them
for (i in seq_along(files$name)) {
  i_dir <- files[i, ] # Select directory
    try({

      # Download files from the directory to the specified path
      drive_download(
        as_id(i_dir$id[1]),
        path = paste0('/content/folder/planet/utm4/2018/', i_dir$name[1])
      )
    })
  }

stopCluster(cl)

# A dribble: 1 × 3
  name        id                                drive_resource   
  <chr>       <drv_id>                          <list>           
1 images_2018 1PndP3cbvQNabol-1-mbRgnsUE-jp8FhD <named list [34]>


Streaming output truncated to the last 5000 lines.
File downloaded:

• 2247_planet.tif <id: 1jJPt80ipMoZn-kdAEgVuhjJeT8kea0ak>

Saved locally as:

• /content/folder/planet/utm4/2018/2247_planet.tif

File downloaded:

• 897_planet.tif <id: 1jPu9uov0Q45RcylJDOpmuKoY_8KH_MTS>

Saved locally as:

• /content/folder/planet/utm4/2018/897_planet.tif

File downloaded:

• 1728_planet.tif <id: 1jTIfVi8-d_Eo_gGY_Szno-eE7FToVmO8>

Saved locally as:

• /content/folder/planet/utm4/2018/1728_planet.tif

File downloaded:

• 1217_planet.tif <id: 1jXUShNuDr2PDgApVz012Dhu-UUW1sknz>

Saved locally as:

• /content/folder/planet/utm4/2018/1217_planet.tif

File downloaded:

• 2009_planet.tif <id: 1jfZewGL77d6M5CN2GBqbNo_66TEzXK-v>

Saved locally as:

• /content/folder/planet/utm4/2018/2009_planet.tif

File downloaded:

• 1364_planet.tif <id: 1juOxwMKaIXfEU3pwWZA1ZP-AofDWPYKD>

Saved locally as:

• /content/folder/planet/utm4/2018/1364_planet.tif

File downloaded:

• 1761_planet.tif <id: 1jzZUDXfJ42Fq15kf3o42V

In [5]:
cl <- makeCluster(8)

# planet utm4 2019
# https://drive.google.com/drive/folders/1lZW5RHiJR6nsx5z1PCDy4f0tAU1OyUsv?usp=drive_link
jp_folder = "1lZW5RHiJR6nsx5z1PCDy4f0tAU1OyUsv"
folder_id = drive_get(as_id(jp_folder)) # get folder ID
print(folder_id) #print folder ID

# Find files in the folder
files = drive_ls(folder_id)

# Loop through directories and download files inside them
for (i in seq_along(files$name)) {
  i_dir <- files[i, ] # Select directory
    try({

      # Download files from the directory to the specified path
      drive_download(
        as_id(i_dir$id[1]),
        path = paste0('/content/folder/planet/utm4/2019/', i_dir$name[1])
      )
    })
  }


stopCluster(cl)

# A dribble: 1 × 3
  name        id                                drive_resource   
  <chr>       <drv_id>                          <list>           
1 images_2019 1lZW5RHiJR6nsx5z1PCDy4f0tAU1OyUsv <named list [34]>


Streaming output truncated to the last 5000 lines.
• 2247_planet.tif <id: 1MXXzUtHCpoGjKuYwrJnK-0qwQrwCc5OC>

Saved locally as:

• /content/folder/planet/utm4/2019/2247_planet.tif

File downloaded:

• 897_planet.tif <id: 1Me5ZiydEX3pwPjs4S9o1R8VbxgUmzpW3>

Saved locally as:

• /content/folder/planet/utm4/2019/897_planet.tif

File downloaded:

• 1728_planet.tif <id: 1MqIqO1GIyxeV8t9nuUPc41JRL0f01wai>

Saved locally as:

• /content/folder/planet/utm4/2019/1728_planet.tif

File downloaded:

• 1217_planet.tif <id: 1MuPBLqbAZjh-H0QqL44buFQVaqNWdcQG>

Saved locally as:

• /content/folder/planet/utm4/2019/1217_planet.tif

File downloaded:

• 2009_planet.tif <id: 1Mxfzc5OPsqjdrp-OBfivywQMleUtw5hK>

Saved locally as:

• /content/folder/planet/utm4/2019/2009_planet.tif

File downloaded:

• 1364_planet.tif <id: 1N-tNnt_efKYFYcb5ZbT19CBUEv4RJQAX>

Saved locally as:

• /content/folder/planet/utm4/2019/1364_planet.tif

File downloaded:

• 1761_planet.tif <id: 1NB-UtOjVfs_cFxIES3v1p7_an-iTgpQU>

Save

In [6]:
cl <- makeCluster(8)

# planet utm4 2020
# https://drive.google.com/drive/folders/1lVOra2Kk0CNtv42zCKe5zC5kO-hYUbiT?usp=drive_link
jp_folder = "1lVOra2Kk0CNtv42zCKe5zC5kO-hYUbiT"
folder_id = drive_get(as_id(jp_folder)) # get folder ID
print(folder_id) #print folder ID

# Find files in the folder
files = drive_ls(folder_id)

# Loop through directories and download files inside them
for (i in seq_along(files$name)) {
  i_dir <- files[i, ] # Select directory
    try({

      # Download files from the directory to the specified path
      drive_download(
        as_id(i_dir$id[1]),
        path = paste0('/content/folder/planet/utm4/2020/', i_dir$name[1])
      )
    })
  }

stopCluster(cl)

# A dribble: 1 × 3
  name        id                                drive_resource   
  <chr>       <drv_id>                          <list>           
1 images_2020 1lVOra2Kk0CNtv42zCKe5zC5kO-hYUbiT <named list [34]>


Streaming output truncated to the last 5000 lines.
File downloaded:

• 2247_planet.tif <id: 1-CcgHlXPG2ZfHWhjkITPiBJ2fYVRi4WN>

Saved locally as:

• /content/folder/planet/utm4/2020/2247_planet.tif

File downloaded:

• 897_planet.tif <id: 1-D0rb-vJvZTJoOTLWta_lH7kFsBboZiA>

Saved locally as:

• /content/folder/planet/utm4/2020/897_planet.tif

File downloaded:

• 1728_planet.tif <id: 1-D7WHXZZy_1muxPA74pQ_XZLmotXTX24>

Saved locally as:

• /content/folder/planet/utm4/2020/1728_planet.tif

File downloaded:

• 1217_planet.tif <id: 1-EVPIB5jQWmVIWc-johjzMv_D0SSYOqr>

Saved locally as:

• /content/folder/planet/utm4/2020/1217_planet.tif

File downloaded:

• 2009_planet.tif <id: 1-GB1JOHW5z_fav75__CDoCy0Z1DkNBbX>

Saved locally as:

• /content/folder/planet/utm4/2020/2009_planet.tif

File downloaded:

• 1364_planet.tif <id: 1-Jo69-bd6wxKo8SHEuvaYOLa78XpBY1f>

Saved locally as:

• /content/folder/planet/utm4/2020/1364_planet.tif

File downloaded:

• 1761_planet.tif <id: 1-L3LHGimrMQcvUoGeXV4g

In [ ]:
cl <- makeCluster(8)

# planet utm4 dem
# https://drive.google.com/drive/folders/1Uzp3U02kGvhldb6HnKwIz2SaQY1KeHHb?usp=drive_link
jp_folder = "1Uzp3U02kGvhldb6HnKwIz2SaQY1KeHHb"
folder_id = drive_get(as_id(jp_folder)) # get folder ID
print(folder_id) #print folder ID

# Find files in the folder
files = drive_ls(folder_id)

# Loop through directories and download files inside them
for (i in seq_along(files$name)) {
  i_dir <- files[i, ] # Select directory
    try({

      # Download files from the directory to the specified path
      drive_download(
        as_id(i_dir$id[1]),
        path = paste0('/content/folder/planet/utm4/dem/', i_dir$name[1])
      )
    })
  }

stopCluster(cl)

In [ ]:
# # planet utm5 2018
# # https://drive.google.com/drive/folders/1P9vfB9MCjMHvn3mwHFNZu4D3qGq-Js-Z?usp=drive_link
# jp_folder = "1P9vfB9MCjMHvn3mwHFNZu4D3qGq-Js-Z"
# folder_id = drive_get(as_id(jp_folder)) # get folder ID
# print(folder_id) #print folder ID

# # Find files in the folder
# files = drive_ls(folder_id)

# # Loop through directories and download files inside them
# for (i in seq_along(files$name)) {
#   i_dir <- files[i, ] # Select directory
#     try({

#       # Download files from the directory to the specified path
#       drive_download(
#         as_id(i_dir$id[1]),
#         path = paste0('/content/folder/planet/utm5/2018/', i_dir$name[1])
#       )
#     })
#   }

# # planet utm5 2019
# # https://drive.google.com/drive/folders/1P6pqHrshe3k6kEvQpfRyPAgfAFQLUoeS?usp=drive_link
# jp_folder = "1P6pqHrshe3k6kEvQpfRyPAgfAFQLUoeS"
# folder_id = drive_get(as_id(jp_folder)) # get folder ID
# print(folder_id) #print folder ID

# # Find files in the folder
# files = drive_ls(folder_id)

# # Loop through directories and download files inside them
# for (i in seq_along(files$name)) {
#   i_dir <- files[i, ] # Select directory
#     try({

#       # Download files from the directory to the specified path
#       drive_download(
#         as_id(i_dir$id[1]),
#         path = paste0('/content/folder/planet/utm5/2019/', i_dir$name[1])
#       )
#     })
#   }

# # planet utm5 2020
# # https://drive.google.com/drive/folders/1P4-Y7sSdQiIDWda19uTHX0KaYsULcDUI?usp=drive_link
# jp_folder = "1P4-Y7sSdQiIDWda19uTHX0KaYsULcDUI"
# folder_id = drive_get(as_id(jp_folder)) # get folder ID
# print(folder_id) #print folder ID

# # Find files in the folder
# files = drive_ls(folder_id)

# # Loop through directories and download files inside them
# for (i in seq_along(files$name)) {
#   i_dir <- files[i, ] # Select directory
#     try({

#       # Download files from the directory to the specified path
#       drive_download(
#         as_id(i_dir$id[1]),
#         path = paste0('/content/folder/planet/utm5/2020/', i_dir$name[1])
#       )
#     })
#   }

# # planet utm5 dem
# # https://drive.google.com/drive/folders/1DRk7b6lKEtihFlg3XXXwmePwNlFIKlA7?usp=drive_link
# jp_folder = "1DRk7b6lKEtihFlg3XXXwmePwNlFIKlA7"
# folder_id = drive_get(as_id(jp_folder)) # get folder ID
# print(folder_id) #print folder ID

# # Find files in the folder
# files = drive_ls(folder_id)

# # Loop through directories and download files inside them
# for (i in seq_along(files$name)) {
#   i_dir <- files[i, ] # Select directory
#     try({

#       # Download files from the directory to the specified path
#       drive_download(
#         as_id(i_dir$id[1]),
#         path = paste0('/content/folder/planet/utm5/dem/', i_dir$name[1])
#       )
#     })
#   }

In [ ]:
# export utm4 stack data to here
# https://drive.google.com/drive/folders/1khRCGAOhrUQJXJ5kx4qwL_GFSHl0aFWS?usp=drive_link
